In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.DEBUG)

In [2]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# 限定只使用1块GPU卡
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [4]:
from keras.datasets import fashion_mnist
(train_x, train_y), (test_x, test_x) = fashion_mnist.load_data()

def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    features = {"x": features}
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features = {"x": features}
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

Using TensorFlow backend.


In [5]:
my_feature_columns = [tf.contrib.layers.real_valued_column("x")]
classifier = tf.estimator.LinearRegressor(feature_columns=my_feature_columns)

# Build 2 hidden layer DNN with 10, 10 units respectively.
# classifier = tf.estimator.DNNClassifier(
#     feature_columns=my_feature_columns,
#     # Two hidden layers of 10 nodes each.
#     hidden_units=[10, 10],
#     # The model must choose between 3 classes.
#     n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_master': '', '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4e17068f98>, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmp3t7o63bb', '_keep_checkpoint_max': 5, '_task_id': 0, '_is_chief': True, '_tf_random_seed': None, '_service': None, '_num_worker_replicas': 1, '_task_type': 'worker'}


In [6]:
BATCH_SIZE = 4
EPOCHS = 1000

# Train the Model.
classifier.train(input_fn=lambda:train_input_fn(train_x, train_y, BATCH_SIZE), steps=EPOCHS)

# Evaluate the model.
eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(test_x, test_y, BATCH_SIZE))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

DEBUG:tensorflow:Transforming feature_column _RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None).
INFO:tensorflow:Create CheckpointSaverHook.


InvalidArgumentError: Input to reshape is a tensor with 3136 values, but the requested shape has 4
	 [[Node: linear/linear_model/x/Reshape = _MklReshape[T=DT_FLOAT, Tshape=DT_INT32, _kernel="MklOp", _device="/job:localhost/replica:0/task:0/device:CPU:0"](linear/linear_model/x/ToFloat, linear/linear_model/x/Reshape/shape, DMT/_2, DMT/_3)]]

Caused by op 'linear/linear_model/x/Reshape', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/